In [1]:
# declaraciones de librerías
using DataStructures
#include("mcmf.jl")
type arcoMCMF
    origen::Int64
    destino::Int64
    flow::Int64
    capacidad::Int64
    cost::Int64
    rev::Int64  # index of reverse edge
end
type gMCMF
    nVertices::Int64
    nArcos::Array{Int64,1}
    adj::Array{arcoMCMF,2}  #adjacency
end

In [2]:
"""
Rutina para generar instancias
"""
function generarGrafoSparse(n::Int64,sp::Float64,maxValue::Int64,maxCost::Int64)
    a=arcoMCMF[]
    for i in 1:n
        for j in i+1:n
            if rand() <= 2.0*sp
                if randn() < 0.0
                    push!(a,arcoMCMF(i,j,0,rand(1:maxValue),rand(1:maxCost),0))
                else
                    push!(a,arcoMCMF(j,i,0,rand(1:maxValue),rand(1:maxCost),0))
                end
            end
        end
    end
    nArcos=zeros(Int64,n)
    for arco in a
        nArcos[arco.origen] += 1
        nArcos[arco.destino] += 1
    end
    adj = Array{arcoMCMF}(n,maximum(nArcos))
    nArcos=zeros(Int64,n)
    for arco in a
        u=arco.origen
        v=arco.destino
        nArcos[u] += 1
        nArcos[v] += 1
        adj[u,nArcos[u]]=arcoMCMF(u,v,0,arco.capacidad,arco.cost,nArcos[v])
        adj[v,nArcos[v]]=arcoMCMF(v,u,0,0,arco.cost,nArcos[u])
    end
    return gMCMF(n,nArcos,adj)
end

"""
Rutina para resolver el problema de maxflow. Otorga una solución inicial y calcula el flujo máximo
"""
function GTMaxFlow!(g::gMCMF,s::Int64,t::Int64)
    #inicialización
    for i in 1:G.nVertices
        for j in 1:G.nArcos[i]
            G.adj[i,j].flow=0
        end
    end
    maxflow=0
    label=zeros(Int64,g.nVertices)
    label[s]=g.nVertices
    exflow=PriorityQueue(Int64,Int64,Base.Order.Reverse)
    for i in 1:g.nVertices
        enqueue!(exflow,i,0)
    end
    for i in 1:g.nArcos[s]
        exflow[g.adj[s,i].destino]=g.adj[s,i].capacidad
        g.adj[s,i].flow=g.adj[s,i].capacidad
        g.adj[ g.adj[s,i].destino , g.adj[s,i].rev ].flow = 0 - g.adj[s,i].capacidad
    end
    node,value = peek(exflow)
    while value>0
        done=false
        #push
        # set γ=min{excess(node),u_f(e)} where v is the source of e and e is (u,v) and Ψ(u)=Ψ(v)+1
        # aument f along e by γ
        for c in 1:g.nArcos[node]
            if (done==false) && (label[node]>label[g.adj[node,c].destino]) && (g.adj[node,c].flow<g.adj[node,c].capacidad)
                #podemos hacer push de flow
                amount=min(value,g.adj[node,c].capacidad-g.adj[node,c].flow)
                #println("push de flow entre ",node," y ",g.adj[node,c].destino,"\t",amount)
                #actualizamos
                exflow[node] = value - amount
                if g.adj[node,c].destino == t
                    maxflow += amount
                else
                    if g.adj[node,c].destino != s
                        exflow[g.adj[node,c].destino] += amount
                    end
                end
                g.adj[node,c].flow += amount
                g.adj[g.adj[node,c].destino,g.adj[node,c].rev].flow -= amount
                done=true
            end
        end
        #relabel
        if done==false
        #set Ψ(v)= 1 + min{ ψ(w) : (v,w)∈δ^+_G_f(v)}
            minvalor=g.nVertices*2
            for c in 1:g.nArcos[node]
                if (g.adj[node,c].flow<g.adj[node,c].capacidad) && minvalor>label[g.adj[node,c].destino]
                    minvalor=label[g.adj[node,c].destino]
                end
            end
            minvalor += 1
            #println("update label ",node," de ",label[node]," a ",minvalor)
            if minvalor<=label[node]
                println("gazapo\n")
                return -1
            else
                label[node]=minvalor
            end
        end
        node,value = peek(exflow)
    end
    return maxflow
end

GTMaxFlow!

In [3]:
G=generarGrafoSparse(10,0.4,1000,1000)
#println(G)
println("sol: ",GTMaxFlow!(G,1,2))

sol: 843


# Min cost max flow

El problema de min cost max flow es una extensión natural del problema de flujo máximo que tiene en cuenta los costes de transporte. El objetivo será ahora transportar la máxima cantidad de flujo posible (o la cantidad de flujo necesaria) intentando minimizar los costos de transporte asociados a cada arco.

Aparte de su interés como problema práctico, el modelo generaliza a otros problemas que han sido extensamente estudiados en la literatura como el problema de asignación y el problema de transporte (problema de Hitchcock).

Mostraremos un algoritmo aunque usaremos dos algoritmos diferentes para resolver el problema y aprovecharemos para tratar dos temas secundarios (importar archivos con código en Julia y ejecutar código programado en C).

# Algoritmo de cancelación de ciclos 

Uno de los algoritmos más simples tanto en idea, implementación y uso para el problema se basa en el siguiente concepto: <br><br>

<center><b>"Si en el grafo residual de una solución de flujo máximo existe un circuito de costo no negativo, la solución actual no es óptima y puede mejorarse redirigiendo el flujo según indica el circuito."</b></center>

Este circuito marca una reconducción de flujo (un cambio) que, sin variar el flujo total que llega al destino, disminuye el coste. 

De forma esquemática, el algoritmo resultante de usar esta propiedad sería:

<b>P1. Inicialización:</b> Encontramos un flujo máximo

<b>P2. Buscar ciclos:</b> Buscamos ciclos en el grafo residual (p.ej. mediante el algoritmo de Bellman-Ford visto con anterioridad, o resolviendo un problema conocido como el "mean cycle problem", lo que reporta una algoritmo fuertemente polinomial)

<b>P3. Update:</b> Si no hay ciclos de costo negativo fin, si no actualizar el grafo residual.

<b>P4. Recalcular:</b> Ir a paso 2.

El funcionamiento del algoritmo puede visualizarse en: https://www-m9.ma.tum.de/graph-algorithms/flow-cycle-cancelling/index_en.html

In [10]:
function meanCycle(g::gMCMF,s::Int64)

end
function cycleCancelling!(g::gMCMF,s::Int64,t::Int64)
    #paso 1. flujo maximo
    GTMaxFlow!(g,s,t)
    #loop
    while true
        break
    end
end

cycleCancelling! (generic function with 1 method)

In [11]:
cycleCancelling!(G,1,2)

# Llamando a una rutina escrita en C (scaling algorithm de Goldberg)

En ocasiones un procedimiento o rutina ya existe en otro lenguaje (en nuestra área, típicamente en C o en Fortran) y uno quiere hacer uso del mismo en vez de entrar en detalles de implementación.

Para mostrar cómo implementar y utilizar rutinas escritas en C, el repositorio incluye, además de las hojas de Jupyter, dos archivos de C (un archivo de declaración types_cs2.h y un archivo de código cs2.c) que serán utilizados para el ejemplo.

Primero debemos compilar el archivo de código como una librería compartida utilizando la siguiente orden en el terminal:

gcc -Wall -shared -o libcs2.so -lm -fPIC cs2.c

In [4]:
run(`gcc -Wall -shared -o libcs2.so -lm -fPIC cs2.c`)

cs2.c: In function ‘parse’:
cs2.c:431:9: warning: unused variable ‘err_no’ [-Wunused-variable]
         err_no;                 /* no of detected error */
         ^
cs2.c:428:9: warning: unused variable ‘pr_type’ [-Wunused-variable]
         pr_type[3];             /* for reading type of the problem */
         ^
cs2.c:427:9: warning: unused variable ‘in_line’ [-Wunused-variable]
 char    in_line[MAXLINE],       /* for reading input line */
         ^
cs2.c:424:9: warning: unused variable ‘no_alines’ [-Wunused-variable]
         no_alines=0,            /* no of arc-lines */
         ^
cs2.c:423:9: warning: unused variable ‘no_nlines’ [-Wunused-variable]
         no_nlines=0,            /* no of node lines */
         ^
cs2.c:422:9: warning: unused variable ‘no_plines’ [-Wunused-variable]
         no_plines=0,            /* no of problem-lines */
         ^
cs2.c:421:9: warning: unused variable ‘no_lines’ [-Wunused-variable]
 long    no_lines=0,             /* no of current input line 

Esta librería tiene dos funciones importantes para nosotros, una es un ejemplo muy simple de una rutina

int simpleprint()
{
  printf("imprimo\n");
}

Que básicamente imprime la palabra imprimo. Para llamar rutina es suficiente con invocar la orden ccall con el nombre de la función, la localización de la librería, el tipo de retorno (Int32) y la lista de parámetros de entrada (en este caso vacía).

In [5]:
ccall((:simpleprint,"/home/jovyan/HerramientasComputacionalesDeAnaliticaCuantitativa/libcs2.so"),Int32,())

imprimo


8

# Utilizando una implementación de cost scaling programada en C

Vamos a mostrar cómo utilizar un algoritmo de cost scaling desarrollado por Goldberg para el problema de max-flow mincost.

Se trata de un algoritmo difícil de implementar que requiere mucho cuidado para obtener implementaciones verdaderamente eficientes (esto es, implementaciones que superen los resultados obtenidos por algoritmos más simples o por el uso de Cplex) y por tanto no intentaremos analizar el algoritmo sino analizar cómo aprovechar el código. 

Nótese que Goldberg publicó una implementación de su algoritmo que utlizaremos aquí únicamente para fines de evaluación (por lo que estrictamente nos estamos saltando ligeramente las leyes de copyright porque el código no es libre. Debido a que ya no está disponible en la página asociada a la empresa que distrubuía comercial ese código y que parece que ya no comercialicen el código, lo consideraré como "abandonware").  

El código de C tiene dos archivos y ha sido retocado lo mínimo posible (por lo que es un "pequeño caos"). La función que llama el código es la función: <b>functMain</b>

Esta función toma múltiples variables de entrada y salida que discutiremos a continuación:

int InN: Número de vértices
int InM: Número de arcos
int countSP: Número de vertices origen y destino (el código acepta varios orígenes y destinos)
int* sP: Puntero a los vértices origen y destino
int* sPV: Puntero a las demandas de los vértices origen y destino (si el valor es negativo, se trata de demanda
int* or: Puntero con la lista de orígenes de los arcos del grafo
int* des: Puntero con la lista de orígenes de los arcos del grafo
int* lower: Puntero con la cantidad mínima de flujo necesario en el arco
int* upper: Puntero con la cantidad máxima de flujo por el arco 
int* costOut: Puntero con el costo por unidad que pasa por el arco

Retorna dos elementos:

int* flowRetorno: Puntero con los flujos en cada arco
int* totFlowRetorno: "Puntero"(en realidad es una variable) para guardar el valor del costo de la solución

Los punteros en Julia se anotan como matrices normales y se indica en Julia que C los quiere tratar como variables (Nótese que int* totFlowRetorno es una variable pero en C para que cambie el valor de una variable la tenemos que pasar como un puntero, que para Julia será una matriz de 1x1.

Veamos ahora el código de un ejemplo simple:

In [8]:
function cs2wrapper(g::gMCMF,s::Int64,t::Int64)
  InN::Int32
  InM::Int32
  countSP::Int32
  #,sP,sPV,or,des,lower,upper,costOut,flowRetorno,&totFlowRetorno
  InN=g.nVertices
  InM=sum(g.nArcos)/2
  countSP=2
  sp=zeros(Int32,countSP)
  spV=zeros(Int32,countSP)
  sp[1]=s
  sp[2]=t
  maxFlow=GTMaxFlow!(g,s,t)
  println("maxFlow: ",maxFlow)
  spV[1]=maxFlow
  spV[2]=0-maxFlow
  count=1
  or=zeros(Int32,InM)
  des=zeros(Int32,InM)
  lower=zeros(Int32,InM)
  upper=zeros(Int32,InM)
  costOut=zeros(Int32,InM)
  flowRetorno=zeros(Int32,InM)
  totalFlowretorno=zeros(Int32,1) #es el int que hay que pasar como un puntero
  for i in 1:g.nVertices
      for j in 1:g.nArcos[i]
          if g.adj[i,j].capacidad>0
              or[count]=g.adj[i,j].origen
              des[count]=g.adj[i,j].destino
              lower[count]=0
              upper[count]=g.adj[i,j].capacidad
              costOut[count]=g.adj[i,j].cost
              count += 1
          end
      end
  end
  if count<InM
      println("ALGO MALO ->",count,"\t",InM)
  end
  println("******")
  println("inN: ",InN)
  println("inM: ",InM)
  println("sp: ",sp)
  println("spV: ",spV)
  println("or: ",or)
  println("des: ",des)
  println("lower: ",lower)
  println("upper: ",upper)
  println("costOut: ",costOut)
  ccall((:functMain,"//home/jovyan/HerramientasComputacionalesDeAnaliticaCuantitativa/libcs2.so"),Int32,(
          Int32, #InN
          Int32, #InM
          Int32, #countSP
          Ptr{Cint}, #sP
          Ptr{Cint}, #sPV
          Ptr{Cint}, #or
          Ptr{Cint}, #des
          Ptr{Cint}, #lower
          Ptr{Cint}, #upper
          Ptr{Cint}, #costOut
          Ptr{Cint}, #flowRetorno
          Ptr{Cint}, #totalFlowretorno
          ),
          InN,InM,countSP,sp,spV,or,des,lower,upper,costOut,flowRetorno,totalFlowretorno)
  #println("******")
  println(flowRetorno)
  println(totalFlowretorno)
  return totalFlowretorno[1]
end


"""
function cs2wrapper(a::Int64,b::Int64,c::Int64)
    InN::Int32
    InM::Int32
    countSP::Int32
    #,sP,sPV,or,des,lower,upper,costOut,flowRetorno,&totFlowRetorno
    InN=a
    InM=b
    countSP=c
    sp=zeros(Int32,countSP)
    sp[1]=1
    sp[2]=6
    spV=zeros(Int32,countSP)
    spV[1]=10
    spV[2]=(-10)
    or=zeros(Int32,InM)
    or[1]=1
    or[2]=1
    or[3]=2
    or[4]=3
    or[5]=5
    or[6]=5
    or[7]=4
    or[8]=4
    des=zeros(Int32,InM)
    des[1]=2
    des[2]=3
    des[3]=3
    des[4]=5
    des[5]=4
    des[6]=6
    des[7]=2
    des[8]=6
    lower=zeros(Int32,InM)
    lower[1]=0
    lower[2]=0
    lower[3]=0
    lower[4]=0
    lower[5]=0
    lower[6]=0
    lower[7]=0
    lower[8]=0
    upper=zeros(Int32,InM)
    upper[1]=4
    upper[2]=8
    upper[3]=5
    upper[4]=10
    upper[5]=8
    upper[6]=8
    upper[7]=8
    upper[8]=8
    costOut=zeros(Int32,InM)
    costOut[1]=1
    costOut[2]=5
    costOut[3]=0
    costOut[4]=1
    costOut[5]=0
    costOut[6]=9
    costOut[7]=1
    costOut[8]=1    
    flowRetorno=zeros(Int32,InM)
    totalFlowretorno=Array{Int32}(1) #es el int que hay que pasar como un puntero
    println(InN," ",InM)
    println("******")
    ccall((:functMain,"/home/jovyan/HerramientasComputacionalesDeAnaliticaCuantitativa/libcs2.so"),Int32,(
            Int32, #InN
            Int32, #InM
            Int32, #countSP
            Ptr{Cint}, #sP
            Ptr{Cint}, #sPV
            Ptr{Cint}, #or
            Ptr{Cint}, #des
            Ptr{Cint}, #lower
            Ptr{Cint}, #upper
            Ptr{Cint}, #costOut
            Ptr{Cint}, #flowRetorno
            Ptr{Cint}, #totalFlowretorno
            ),
            InN,InM,countSP,sp,spV,or,des,lower,upper,costOut,flowRetorno,totalFlowretorno)
    println("******")
    println(flowRetorno)
    println(totalFlowretorno)
end
"""


Use "local InN::Int32" instead.

Use "local InM::Int32" instead.

Use "local countSP::Int32" instead.


"function cs2wrapper(a::Int64,b::Int64,c::Int64)\n    InN::Int32\n    InM::Int32\n    countSP::Int32\n    #,sP,sPV,or,des,lower,upper,costOut,flowRetorno,&totFlowRetorno\n    InN=a\n    InM=b\n    countSP=c\n    sp=zeros(Int32,countSP)\n    sp[1]=1\n    sp[2]=6\n    spV=zeros(Int32,countSP)\n    spV[1]=10\n    spV[2]=(-10)\n    or=zeros(Int32,InM)\n    or[1]=1\n    or[2]=1\n    or[3]=2\n    or[4]=3\n    or[5]=5\n    or[6]=5\n    or[7]=4\n    or[8]=4\n    des=zeros(Int32,InM)\n    des[1]=2\n    des[2]=3\n    des[3]=3\n    des[4]=5\n    des[5]=4\n    des[6]=6\n    des[7]=2\n    des[8]=6\n    lower=zeros(Int32,InM)\n    lower[1]=0\n    lower[2]=0\n    lower[3]=0\n    lower[4]=0\n    lower[5]=0\n    lower[6]=0\n    lower[7]=0\n    lower[8]=0\n    upper=zeros(Int32,InM)\n    upper[1]=4\n    upper[2]=8\n    upper[3]=5\n    upper[4]=10\n    upper[5]=8\n    upper[6]=8\n    upper[7]=8\n    upper[8]=8\n    costOut=zeros(Int32,InM)\n    costOut[1]=1\n    costOut[2]=5\n    costOut[3]=0\n    costOu

In [9]:
cs2wrapper(G,1,2)

maxFlow: 843
******
inN: 10
inM: 39
sp: Int32[1,2]
spV: Int32[843,-843]
or: Int32[1,2,2,2,3,3,3,3,3,4,4,5,5,5,6,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,9,9,9,9,9,9,10,10,10]
des: Int32[8,1,5,8,1,4,5,7,10,1,2,6,7,8,1,2,3,7,8,9,10,1,2,4,8,9,10,3,4,10,1,2,3,4,5,10,2,4,5]
lower: Int32[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
upper: Int32[843,875,820,880,47,675,385,945,362,558,542,167,7,780,120,797,126,983,956,567,260,753,249,356,92,658,562,853,252,584,407,120,561,93,73,838,2,988,476]
costOut: Int32[388,632,922,320,894,946,380,202,180,759,863,508,774,165,376,249,41,190,647,541,541,485,420,653,914,171,286,296,869,352,547,773,857,433,874,730,515,970,86]
Int32[843,0,0,0,0,0,0,422,0,0,305,167,0,0,0,167,0,0,0,0,0,0,249,0,0,173,0,422,252,169,0,120,0,53,0,0,2,0,167]
Int32[1470614]


1470614

# ¿Qué se nos ha quedado en el tintero?

La respuesta es sí, lógicamente. Entre las cosas que podrían haberse visto pero no ha dado tiempo:

* <b>Branch and bound</b>. Una buena manera de empezar sería a partir del problema de las n-reinas visto con anterioridad. La interelación entre branch and bound y programación dinámica también es un tópico que podríamos haber estudiado y analizado.
* <b>Benders decomposition</b>. Aunque no se ha visto como implementación, sí se ha tratado del tema durante la descripción de JuMP y los planos cortantes.
* <b>Branch and price y branch, cut and price</b>. Aunque forman parte del toolbox, es un algoritmo difícil de programar y optimizar.
* <b>Asignación, transporte y matching</b>. Hemos visto un caso más general que los anteriores (excepto matching no bipartito), pero estos problemas acostumbran a tratarse como subproblemas y no se han visto en el curso
* <b>Heurísticas y metaheurísticas</b>.
* <b>Programación no lineal</b>
* Limpieza y estructura de código. Específicamente hay tres problemas graves en los códigos desarrollados: <b>(1)</b> uso de castellano e inglés indistintamente; <b>(2)</b> cambios en la notación (uso de mayúscula y minúscula indistintamente); y <b>(3)</b> mala documentación.